In [ ]:
%matplotlib inline
import numpy

import matplotlib.pyplot as plt

In [ ]:
# Generate data...
numpy.random.seed(0)
x = numpy.random.rand(1024, 2)

y = numpy.zeros(1024, dtype=int)
edge = 0.5 * numpy.sqrt(0.5)
y[numpy.logical_and(x>(0.5-edge), x<(0.5+edge)).all(axis=1)] = 1



# Visualise...
plt.figure(figsize=(16,16))
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.axis('off')

plt.scatter(x[y==0,0], x[y==0,1], c='b', edgecolors='black')
plt.scatter(x[y==1,0], x[y==1,1], c='r', edgecolors='black')

plt.savefig('ada_data.png', bbox_inches='tight')
plt.show()

In [ ]:
class DecStump:
    """Decision stump, two classes only, except it selects a number of random 
    directions and find the best split (gini); very slightly smarter
    in other words."""
    
    def __init__(self, x, y, w = None, dirs = 16):
        """y must be interger, containing only 0 and 1. w is the weights."""
        if w is None:
            w = numpy.ones(y.shape[0])
    
        # Parameters...
        self.beta = numpy.zeros(x.shape[1])
        self.beta[0] = 1.0
        self.offset = 0.0
        self.left = 0.5 # Probability of class 1 (vs 0)
        self.right = 0.5 # "
    
        self.gini = numpy.inf
        
        total = w.sum()
        
        # Find a good split...
        for _ in range(dirs):
            # Random direction...
            beta = numpy.random.standard_normal(x.shape[1])
            beta /= numpy.sqrt(numpy.square(beta).sum())
            
            # Project data...
            px = (x * beta[None,:]).sum(axis=1)
            
            # Sort by projected position...
            order = numpy.argsort(px)
            px = px[order]
            py = y[order]
            pw = w[order]
            
            # Calculate class 0 and class 1 total weight...
            left_w = numpy.maximum(numpy.cumsum(pw), 1e-3)
            left_w0 = numpy.cumsum(pw * (py==0))
            left_w1 = numpy.cumsum(pw * (py==1))
            
            right_w = numpy.maximum(numpy.cumsum(pw[::-1])[::-1], 1e-3)
            right_w0 = numpy.cumsum((pw * (py==0))[::-1])[::-1]
            right_w1 = numpy.cumsum((pw * (py==1))[::-1])[::-1]
            
            # Calculate error for each split point then the combined term...
            left_gini = 1 - numpy.square(left_w0 / left_w) - numpy.square(left_w1 / left_w)
            right_gini = 1 - numpy.square(right_w0 / right_w) - numpy.square(right_w1 / right_w)
            
            gini = left_w[:-1] * left_gini[:-1] + right_w[1:] * right_gini[1:]

            # Find minimum and record if the winner...
            best = numpy.argmin(gini)
            if gini[best] < self.gini:
                self.beta = beta
                self.offset = 0.5 * (px[best] + px[best+1])
                
                self.left = (py[:best+1] * pw[:best+1]).sum() / pw[:best+1].sum()
                self.right = (py[best+1:] * pw[best+1:]).sum() / pw[best+1:].sum()
                
                self.gini = gini[best]
    
    
    def prob(self, x):
        """Given coordinates this returns a 1D array containing the probability of
        each having label 1, as opposed to label 0"""
        ret = numpy.empty(x.shape[0])
        
        ret.fill(self.left)
        ret[(x * self.beta[None,:]).sum(axis=1) > self.offset] = self.right
        
        return ret
    
    
    def __call__(self, x):
        """Calculates the label (0 or 1) of a datamatrix (2D array)"""
        return (self.prob(x)>0.5).astype(int)
    
    
    def errors(self, x, y):
        """Returns a 0/1 array where it's zero when an answer is correct, 1 when it is wrong."""
        ey = self(x)
        return (ey!=y).astype(int)
        

    def visualise(self, sweep_x = None, sweep_y = None):
        """Returns the probability of being class 1 given two sweeps of values,
        typically generated by linspace"""
        if sweep_x is None:
            sweep_x = numpy.linspace(0, 1, 256)
        if sweep_y is None:
            sweep_y = numpy.linspace(0, 1, 300)
        
        coords = numpy.transpose(numpy.meshgrid(sweep_x, sweep_y, indexing='ij')).reshape((-1, 2))

        value = numpy.empty(coords.shape[0])
        value.fill(self.left)
        
        value[(coords * self.beta[None,:]).sum(axis=1) > self.offset] = self.right
        
        value = value.reshape((sweep_y.shape[0], sweep_x.shape[0]))
        return value

    
    
# Verification...
model = DecStump(x, y)
print('Beta = [{} {}]'.format(model.beta[0], model.beta[1]))
print('Offset = {}'.format(model.offset))
print('P(left)=1  = {}'.format(model.left))
print('P(right)=1 = {}'.format(model.right))
print('Gini = {}'.format(model.gini))


plt.figure(figsize=(16,16))
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.axis('off')

vis = model.visualise()
#vis[vis<0.5] = 0.0
#vis[vis>0.5] = 1.0
plt.imshow(vis, vmin=0, vmax=1, cmap='plasma', interpolation='bilinear', origin='lower', extent=(0, 1, 0, 1))

plt.scatter(x[y==0,0], x[y==0,1], c='b', edgecolors='black')
plt.scatter(x[y==1,0], x[y==1,1], c='r', edgecolors='black')

plt.show()

In [ ]:
class AdaBoost:
    """Simple AdaBoost implementation, primarily for visualisation of toy problems."""
    
    def __init__(self, x , y, dirs = 16):
        """Only initalises - need to call add for each model desired."""
        # Record state...
        self.x = x
        self.y = y
        
        self.w = numpy.empty(self.y.shape[0])
        self.w.fill(1.0 / self.y.shape[0])
        
        self.dirs = dirs
        self.models = [] # Pairs of (alpha, DecStump)
    
    
    def add(self):
        """Adds a new model."""
        # Train model...
        model = DecStump(self.x, self.y, self.w, dirs=self.dirs)
        
        # Calculate error...
        errors = model.errors(self.x, self.y)
        error = (errors * self.w).sum() / self.w.sum()
        
        # Calculate alpha...
        error = min(error, 1e-6)
        alpha = numpy.log((1 - error) / error)
        
        # Update weights...
        self.w *= numpy.exp(alpha * errors)
        
        # Record...
        self.models.append((alpha, model))
    
    
    def visualise(self, sweep_x = None, sweep_y = None):
        if sweep_x is None:
            sweep_x = numpy.linspace(0, 1, 256)
        if sweep_y is None:
            sweep_y = numpy.linspace(0, 1, 300)
        
        coords = numpy.transpose(numpy.meshgrid(sweep_x, sweep_y, indexing='ij')).reshape((-1, 2))
        
        y = numpy.zeros(coords.shape[0])
        div = 0.0
        for alpha, model in self.models:
            y += alpha * model(coords)
            div += alpha

        y /= div

        return y.reshape((sweep_y.shape[0], sweep_x.shape[0]))
    
    
    def __len__(self):
        return len(self.models)
